Required imports

In [1]:
import pandas as pd
import re
from typing import Tuple, Dict, FrozenSet
import logging
from datetime import datetime

These are the common functions we will be using the entire code

In [2]:
COLUMNS_TO_DELETE = set()

def parse_redcap_event(event_name: str) -> Tuple[int, int]:
  # Returns the vist number and arm number
  pattern = r'visit_(\d+)_arm_(\d+)'
  matches = re.search(pattern, event_name)
  if matches is None:
    raise Exception(f"Unable to parse the string: \'{event_name}\' as a redcap event, please check the code / clean the data")
  return int(matches.group(1)), int(matches.group(2))

def generate_column_name(visit_number, arm_number, old_column_name) -> str:
  return f"{old_column_name}_vist_{visit_number}_arm_{arm_number}"

def merge_rows(selected_rows):
  # Overall Logic -  Merge the selected rows based on the logic:
  # 1. If the column values are mutually exculusive in all the given rows, just shift it all up
  # 2. If the column values exist in multiple rows, create new columns in the new_dataframe

  # Step 1 - Start by just copying the first row into the new DF
  # Step 2 - Now loop through all the columns in the other rows to see what needs to get done

  # Step 1 - Start by just copying the first row that we mutate and append
  # to the new Dataframe
  new_row = selected_rows.head(1).copy()

  # Step 2 - Now loop through all the columns except the first two since
  # This will be the main checking loop
  for column_name in list(selected_rows.columns)[2:]:
    # print(f"Checking Colmn:{column_name}")
    # Extract the column data
    column_data = list(selected_rows[column_name])

    # Check if its mutually excusive, compute how many values aren't NaN
    not_null_count = 0
    last_not_null_index = None
    for row_index, value in enumerate(column_data):
      if (pd.isna(value) or (value is None)) is False:
        not_null_count += 1
        last_not_null_index = row_index


    # If the count is > 1 we need to create new columns
    if not_null_count > 1:
      # raise NotImplementedError(f"Implement new column creation: Column: {column_name} hhid: {new_row['hhid']}")

      logging.info(column_data)
      logging.info(f"Marked column: {column_name} for moving data")

      COLUMNS_TO_DELETE.add(column_name)

    elif not_null_count == 1:
      logging.info(column_data)
      # Copy the data into the new dataframe
      if last_not_null_index is None:
        raise Exception("Last seen index is none")
      new_row[column_name] = column_data[last_not_null_index]
      logging.info(f"Moved data for column name: {column_name} for hhid: {new_row['hhid']}")
    elif not_null_count == 0:
      continue
    else:
      raise Exception(f"invalid not_null_count value: {not_null_count}")

  # Update the new row

  return new_row


def trim_columns(hhid_list, old_data_frame, new_data_frame):
  new_data_frame = new_data_frame.copy()
  # Get all the rows with the corresponding HHID
  for current_hhid in hhid_list:
    selected_rows = old_data_frame[old_data_frame['hhid'] == current_hhid]

    # Perform moves for all the columns marked for deletion
    new_data_frame = move_data(current_hhid, selected_rows, new_data_frame)

  # Move all the data that is l
  # Drop all the old columns (that gor renamed)
  new_data_frame = new_data_frame.drop(list(COLUMNS_TO_DELETE), axis=1)
  logging.info(f"Deleted old column: {list(COLUMNS_TO_DELETE)}")
  return new_data_frame



def move_data(current_hhid, selected_rows, data_frame):
  # Go through each columns
  for column_name in list(COLUMNS_TO_DELETE):

    # Extract the column data
    column_data = list(selected_rows[column_name])

    for row_index, value in enumerate(column_data):

      # Only perform the check and addition
      if pd.isna(value) is True or (value is None) is True:
        continue

      # Step 1 - Generate the new column name
      event_name = selected_rows.iloc[row_index]['redcap_event_name']
      visit_number, arm_number = parse_redcap_event(event_name)
      new_column_name = generate_column_name(visit_number, arm_number, column_name)


      # Step 2 -  add the data directly as the new column, let pandas handle
      # indexing
      current_row = data_frame[data_frame['hhid'] == current_hhid]

      if new_column_name in data_frame.columns:
        current_row[new_column_name] = value
        logging.info(f"Moved data to a new column: {new_column_name} for hhid: {current_row['hhid']} arm number: {arm_number} and value: {value}")
      else:
        print(f"Generated a new Column name: {new_column_name}")
        #column_df = pd.DataFrame({new_column_name: [value] })
        current_row[new_column_name] = value
        #current_row = pd.concat([current_row, column_df], axis=1)
        logging.info(f"Added a new column: {new_column_name} for hhid: {current_row['hhid']} arm number: {arm_number} and value: {value}")

  return data_frame



In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# Setup Logging
import logging
logging.basicConfig(filename="log_file_test.log",
                            filemode='a',
                            format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                            datefmt='%H:%M:%S',
                            level=logging.DEBUG)

In [5]:
# Load the csv as a pandas dataframe that makes it easy to manipulate and query
# the data (we point out that the header is in row 0)
dataframe_original = pd.read_csv('/root/sandbox/disa/data_v0004.tsv', sep='\t', header=0)



# To be changed later on but we are taking the first 10 rows as our testing data
df = dataframe_original.head(50)

/tmp/ipykernel_2694519/2758161052.py:3: DtypeWarning: Columns (4,17,22,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,104,136,226,227,327,462,492,515,519,520,521,525,544,545,568,569,570,572,573,574,577,578,579,580,581,582,584,585,586,587,588,590,591,592,594,595,599,601,602,603,605,606,607,608,624,626,627,628,629,633,634,635,663,664,667,668,684,700,712,725,726,729,730,731,734,742,743,744,745,746,747,748,749,750,751,752,753,754,756,757,760,761,765,767,768,769,770,773,774,792,793,799,800,801,829,830,832,833,834,836,846,848,849,850,852,853,855,857,864,865,866,868,878,879,890,891,894,898,899,900,901,907,908,909,910,911,912,913,914,915,916,917,919,923,927,929,933,934,939,940,965,966,967,995,1000,1012,1014,1015,1016,1018,1019,1021,1023,1030,1031,1032,1034,1045,1056,1057,1060,1064,1065,1066,1067,1073,1074,1075,1080,1083,1089,1132,1178,1180,1181,1184,1187,1189,1196,1197,1198,1200,1211,1222,1226,1230,1231,1232,1233,1239,1471,1473,1474,1476,1477,1478,1482,1486,1487,148

In [6]:
# Generate a list of all unique hhid's, we will be using these are the means to
# merge the rows

unique_hhids = df['hhid'].unique()

# Create a new dataframe where we store all the data
new_df = pd.DataFrame(columns=df.columns)

# Loop through each of the HHID's and then perform the merging for each hhid
for i in range(len(unique_hhids)):
  current_hhid = unique_hhids[i]
  print(current_hhid)
  # Get the rows corresponding to the given hhid
  selected_rows = df[df['hhid'] == current_hhid]
  # Now merge the rows with the same HHID in the new Dataframe
  new_row = merge_rows(selected_rows)
  print(f"Appending the row for HHID: {new_row['hhid']}")
  new_df = pd.concat([new_df, new_row], axis=0, ignore_index=True)

# Move all the data from columns marked to be deleted into their new columns,
trimmed_df = trim_columns(unique_hhids, df, new_df)

date_time = datetime.now()
str_date_time = date_time.strftime("%d-%m-%Y-%H:%M:%S")

# Now save the new dataframe as a csv
trimmed_df.to_csv(f'/root/sandbox/disa/data_merged_{str_date_time}.tsv', sep="\t", index=False)

AT-GT-KR-KP-2168
Appending the row for HHID: 0    AT-GT-KR-KP-2168
Name: hhid, dtype: object
AT-GT-KR-KP-2169
Appending the row for HHID: 5    AT-GT-KR-KP-2169
Name: hhid, dtype: object
AT-GT-KR-KP-2170
Appending the row for HHID: 9    AT-GT-KR-KP-2170
Name: hhid, dtype: object
AT-GT-KR-KP-2171
Appending the row for HHID: 14    AT-GT-KR-KP-2171
Name: hhid, dtype: object
AT-GT-KR-KP-2172
Appending the row for HHID: 19    AT-GT-KR-KP-2172
Name: hhid, dtype: object
AT-GT-KR-KP-2173
Appending the row for HHID: 24    AT-GT-KR-KP-2173
Name: hhid, dtype: object
AT-GT-KR-KP-2174
Appending the row for HHID: 29    AT-GT-KR-KP-2174
Name: hhid, dtype: object
AT-GT-KR-KP-2175
Appending the row for HHID: 34    AT-GT-KR-KP-2175
Name: hhid, dtype: object
AT-GT-KR-KP-2176
Appending the row for HHID: 39    AT-GT-KR-KP-2176
Name: hhid, dtype: object
AT-GT-KR-KP-2177
Appending the row for HHID: 44    AT-GT-KR-KP-2177
Name: hhid, dtype: object
AT-GT-KR-KP-2178
Appending the row for HHID: 49    AT-GT-KR-KP-

/tmp/ipykernel_2694519/1877440350.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_row[new_column_name] = value
/tmp/ipykernel_2694519/1877440350.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_row[new_column_name] = value
/tmp/ipykernel_2694519/1877440350.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

Generated a new Column name: a10_0300_vist_1_arm_2
Generated a new Column name: a5_land___4_vist_2_arm_1
Generated a new Column name: a5_land___4_vist_2_arm_2
Generated a new Column name: a5_loan_bank_decision___3_vist_2_arm_1
Generated a new Column name: a5_loan_bank_decision___3_vist_2_arm_2
Generated a new Column name: a5_loan_bank_decisio_v_0_vist_2_arm_1
Generated a new Column name: a5_loan_bank_decisio_v_0_vist_2_arm_2
Generated a new Column name: a5_durables_large___1_vist_2_arm_1
Generated a new Column name: a5_durables_large___1_vist_2_arm_2
Generated a new Column name: a5_bigexp_who___999_vist_2_arm_1
Generated a new Column name: a5_bigexp_who___999_vist_2_arm_2
Generated a new Column name: a5_wage_who___888_vist_2_arm_1
Generated a new Column name: a5_wage_who___888_vist_2_arm_2
Generated a new Column name: a10_date_vist_1_arm_1
Generated a new Column name: a10_date_vist_1_arm_2
Generated a new Column name: a5_cashcrop_income_vist_2_arm_1
Generated a new Column name: a5_cash

/tmp/ipykernel_2694519/1877440350.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_row[new_column_name] = value
/tmp/ipykernel_2694519/1877440350.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_row[new_column_name] = value
/tmp/ipykernel_2694519/1877440350.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

Generated a new Column name: a5_livestock_who___999_vist_2_arm_1
Generated a new Column name: a5_livestock_who___999_vist_2_arm_2
Generated a new Column name: a5_bigexp_who___3_vist_2_arm_1
Generated a new Column name: a5_bigexp_who___3_vist_2_arm_2
Generated a new Column name: a10_0400_vist_1_arm_1
Generated a new Column name: a10_0400_vist_1_arm_2
Generated a new Column name: a5_equipment_buss___1_vist_2_arm_1
Generated a new Column name: a5_equipment_buss___1_vist_2_arm_2
Generated a new Column name: a5_loan_shg_decision___4_vist_2_arm_1
Generated a new Column name: a5_loan_shg_decision___4_vist_2_arm_2
Generated a new Column name: a5_healthtreatment_vist_2_arm_1
Generated a new Column name: a5_healthtreatment_vist_2_arm_2
Generated a new Column name: a5_bigexp_who___2_vist_2_arm_1
Generated a new Column name: a5_bigexp_who___2_vist_2_arm_2
Generated a new Column name: a10_1330_vist_1_arm_1
Generated a new Column name: a10_1330_vist_1_arm_2
Generated a new Column name: a5_equipment_

...

/tmp/ipykernel_2694519/1877440350.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_row[new_column_name] = value
/tmp/ipykernel_2694519/1877440350.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_row[new_column_name] = value
/tmp/ipykernel_2694519/1877440350.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

Generated a new Column name: a10_1730_vist_1_arm_1
Generated a new Column name: a10_1730_vist_1_arm_2
Generated a new Column name: a5_livestock_have___3_vist_2_arm_1
Generated a new Column name: a5_livestock_have___3_vist_2_arm_2
Generated a new Column name: a5_cashcrop_vist_2_arm_1
Generated a new Column name: a5_cashcrop_vist_2_arm_2
Generated a new Column name: a10_0530_vist_1_arm_1
Generated a new Column name: a10_0530_vist_1_arm_2
Generated a new Column name: a5_loan_shg_decision_who___2_vist_2_arm_1
Generated a new Column name: a5_loan_shg_decision_who___2_vist_2_arm_2
Generated a new Column name: a5_small_livestock___1_vist_2_arm_1
Generated a new Column name: a5_small_livestock___1_vist_2_arm_2
Generated a new Column name: a5_wage_who___4_vist_2_arm_1
Generated a new Column name: a5_wage_who___4_vist_2_arm_2
Generated a new Column name: a5_visit_community_vist_2_arm_1
Generated a new Column name: a5_visit_community_vist_2_arm_2
Generated a new Column name: a5_nonfarm_who___2_vi

/tmp/ipykernel_2694519/1877440350.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_row[new_column_name] = value
/tmp/ipykernel_2694519/1877440350.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_row[new_column_name] = value
/tmp/ipykernel_2694519/1877440350.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

Generated a new Column name: a10_0500_vist_1_arm_1
Generated a new Column name: a10_0330_vist_1_arm_1
Generated a new Column name: a10_2300_vist_1_arm_1
Generated a new Column name: a10_1600_vist_1_arm_1
Generated a new Column name: a10_2200_vist_1_arm_1
Generated a new Column name: a10_1830_vist_1_arm_1
Generated a new Column name: a10_1230_vist_1_arm_1
Generated a new Column name: a10_1000_vist_1_arm_1
Generated a new Column name: a10_2000_vist_1_arm_1
Generated a new Column name: a10_1900_vist_1_arm_1
Generated a new Column name: a10_1030_vist_1_arm_1
Generated a new Column name: a10_1530_vist_1_arm_1
Generated a new Column name: a10_1300_vist_1_arm_1
Generated a new Column name: a10_interviewer_vist_1_arm_1
Generated a new Column name: a10_0200_vist_1_arm_1
Generated a new Column name: a10_0100_vist_1_arm_1
Generated a new Column name: a10_1130_vist_1_arm_1
Generated a new Column name: a10_2230_vist_1_arm_1
Generated a new Column name: time_use_complete_vist_1_arm_1
Generated a new

/tmp/ipykernel_2694519/1877440350.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_row[new_column_name] = value
/tmp/ipykernel_2694519/1877440350.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_row[new_column_name] = value
/tmp/ipykernel_2694519/1877440350.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [7]:
# # Loop through all the rows skipping the first
      # for row_index, value in enumerate(column_data):

      #   # Only perform the check and addition
      #   if pd.isna(value) is True or (value is None) is True:
      #     continue

      #   # Step 1 - Generate the new column name
      #   event_name = selected_rows.iloc[row_index]['redcap_event_name']
      #   visit_number, arm_number = parse_redcap_event(event_name)
      #   new_column_name = generate_column_name(visit_number, arm_number, column_name)
      #   print(f"Generated a new Column name: {new_column_name}")

      #   # Step 2 -  add the data directly as the new column, let pandas handle
      #   # indexing
      #   # Check if new column name is in the dataframe
      #   if new_column_name in new_row.columns:
      #     new_row[new_column_name] = value
      #     logging.info(f"Moved data to a new column: {new_column_name} for hhid: {new_row['hhid']} arm number: {arm_number} and value: {value}")
      #   else:
      #     column_df = pd.DataFrame({new_column_name: [value] })
      #     new_row = pd.concat([new_row, column_df], axis=1)
      #     logging.info(f"Added a new column: {new_column_name} for hhid: {new_row['hhid']} arm number: {arm_number} and value: {value}")

      # Step 3 - SAVE original column in row dataframe once we finish moving
      # everything out. We will delete these columns later

In [8]:
print(COLUMNS_TO_DELETE)

{'a5_loan_shg', 'a10_0930', 'a10_2100', 'a5_smallexp_who___2', 'a5_wage_input', 'a5_loan_input_decisi_v_2', 'a10_1200', 'a5_cashcrop_who___4', 'a5_cashcrop_input', 'a5_nonfarm', 'a5_loan_shg_decision_who___3', 'a5_equipment_non___0', 'a5_equipment___1', 'a5_fpo', 'a5_loan_bank_decision_who___4', 'a5_cashcrop_who___3', 'a5_interview', 'a5_wage', 'a5_interviewer', 'a5_house___2', 'a5_phone_own___4', 'a5_equipment___0', 'a5_loan_bank_decision_who___1', 'a5_fpo_member', 'a10_0230', 'a5_land_nonagri___1', 'a5_house___4', 'a5_livestock', 'a5_loan_input_decision_who___2', 'a5_nonfarm_income', 'a5_smallexp_who___1', 'a5_loan_bank_decision___999', 'a10_0300', 'a5_land___4', 'a5_loan_bank_decision___3', 'a5_loan_bank_decisio_v_0', 'a5_durables_large___1', 'a5_bigexp_who___999', 'a5_wage_who___888', 'a10_date', 'a5_cashcrop_income', 'a5_livestock_type___4', 'a5_cashcrop_who___1', 'a5_loan_ff_decision_who___2', 'a5_phone_own___3', 'a10_0600', 'a10_1630', 'a5_nonfarm_who___3', 'a5_wage_who___3', 'a

Columns to Check:
- a5_visit_market1
- 